## Loading data

In [18]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

import os
import zipfile
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

--2024-11-04 05:04:40--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.207, 74.125.139.207, 173.194.212.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   218MB/s    in 0.3s    

2024-11-04 05:04:40 (218 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Reading the Data into arrays

In [24]:
from PIL import Image
import numpy as np
x_train = []
y_train = []
x_test = []
y_test = []

cats_dir = os.path.join(train_dir + "/cats")
dogs_dir = os.path.join(train_dir + "/dogs")

i = 0
while i < 800:
  if i % 2 == 0:
    im = Image.open(os.path.join(cats_dir, os.listdir(cats_dir)[i])).convert("RGB")
    im_resized = im.resize((150, 150))
    x_train.append(np.array(im_resized))
    y_train.append(1)
  else:
    im = Image.open(os.path.join(dogs_dir, os.listdir(dogs_dir)[i])).convert("RGB")

    x_train.append(np.array(im_resized))
    y_train.append(0)
  i += 1

while i < 1000:
  if i % 2 == 0:
    im = Image.open(os.path.join(cats_dir, os.listdir(cats_dir)[i])).convert("RGB")
    im_resized = im.resize((150, 150))
    x_test.append(np.array(im_resized))
    y_test.append(1)
  else:
    im = Image.open(os.path.join(dogs_dir, os.listdir(dogs_dir)[i])).convert("RGB")

    x_test.append(np.array(im_resized))
    y_test.append(0)
  i += 1

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

## Beginning to define the model (this is where you come in, I loaded the pretrained model for you)

In [28]:
import tensorflow as tf

pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(150, 150, 3),
    pooling='avg',
    classes=2,
    weights='imagenet'
)

final_model = tf.keras.models.Sequential()
final_model.add(pretrained_model)

for layer in final_model.layers:
  layer.trainable = False

final_model.add(tf.keras.layers.Flatten())
final_model.add(tf.keras.layers.Dense(256, activation = "relu"))
final_model.add(tf.keras.layers.Dense(1, activation = "sigmoid"))

final_model.compile(optimizer='adam',
  loss='binary_crossentropy',
  metrics=['accuracy'])

final_model.fit(x_train, y_train, epochs=3)

Epoch 1/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - accuracy: 0.4579 - loss: 1.5125
Epoch 2/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 77s 3s/step - accuracy: 0.4875 - loss: 0.7052
Epoch 3/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - accuracy: 0.4697 - loss: 0.6953


In [29]:
final_model.evaluate(x_test, y_test)
print("------------------------")

from sre_constants import SUCCESS
probabilities = final_model.predict(x_test)
predictions = np.argmax(probabilities, axis=1)
successes = 0
fails = 0
for index in range(len(probabilities)):
  if predictions[index] == y_test[index]:
    successes += 1
  else:
    fails += 1

print("Accuracy: ", successes * 100 / (successes + fails), "%")

# It's always giving exactly 50% accuracy, so I know something's wrong, but I don't know what.

7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.5000 - loss: 0.6938
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 3s/step
Accuracy:  50.0 %
